# Mandatory imports and utils

In [ ]:
import Control.Monad
import Control.Monad.Primitive

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants (m_epsilon)
import Numeric.MathFunctions.Comparison

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy

In [ ]:
linspace :: (Double,Double) -> Int -> [Double]
linspace (a,b) n = [a + (b - a) * fromIntegral i / fromIntegral n           | i <- [0 .. n]]

logspace :: (Double,Double) -> Int -> [Double]
logspace (a,b) n = [exp $ la + (lb - la) * fromIntegral i / fromIntegral n  | i <- [0 .. n]]
  where la = log a
        lb = log b

-- Generate list of points 
funcPoints f (a,b) n =
  [(x,y) | x <- linspace (a,b) n
         , let y = f x
         , not (isInfinite y)
         , not (isNaN y)
  ]

funcPointsLog f (a,b) n =
  [(LogValue x,y) | x <- logspace (a,b) n
                  , let y = f x
                  , not (isInfinite y)
                  , not (isNaN y)
  ]

-- Genric plotter
plotFunctions funs rng
  = layout_plots .~
      [ toPlot $ plot_lines_values .~ [funcPoints fun rng 1000]
               -- $ plot_lines_style  .~ (line_color .~ opaque color $ def)
               $ def
      | (color,fun) <- cycle [blue,red,green] `zip` funs
      ]
  $ def
  
plotFunctionsLog funs rng
  = layout_plots .~
      [ toPlot $ plot_lines_values .~ [funcPointsLog fun rng 1000]
              -- $ plot_lines_style  .~ (line_color .~ opaque color $ def)
               $ def
      | (color,fun) <- cycle [blue,red,green] `zip` funs
      ]
  $ def

# Incomplete beta

Quick reminder about beta function and (regularized) incomplete beta functions:

Beta function:
$$B(a,b) = \int_0^1 t^{a-1}(1 - t)^{b-1} \,dt $$

Incomplete beta:
$$B(x; a,b) = \int_0^x t^{a-1}(1 - t)^{b-1} \,dt \qquad x \in [0,1]$$

Regularized incomplete beta (from now it'll be referred to simply as incomplete beta)
$$I(x; a,b) = \frac{B(x; a,b)}{B(a,b)}$$


# [Inverse failure to converge!](https://github.com/bos/math-functions/issues/35)

Originally uncovered when plotting roundtrip error of `cumulative . quantile` for beta distribution in log scale.

In [ ]:
-- Bird's eye view
toRenderable
  $ plotFunctions [invIncompleteBeta 7 0.07] (1e-7,1e-6)
-- Closeup
toRenderable
  $ plotFunctions [invIncompleteBeta 7 0.07] (2.8e-7,3e-7)

## [Loss of precision for p=0.5!](https://github.com/bos/math-functions/issues/36)

Again uncovered in plot for `cumulative . quantile` roundtrip error. It manifested itself as sharp spike near `p = 0.5`.

Now how does plot looks like:

In [ ]:
toRenderable
  $ plotFunctions [incompleteBeta 4.5 4.5] (0,1)

Closeup of roundtrip error

In [ ]:
let fun x = let p  = invIncompleteBeta 4.5 4.5 x
                x' = incompleteBeta 4.5 4.5 p
            in x'
toRenderable
  $ let d = 5e-9 
     in plotFunctions [\x -> logBase 10 $ relativeError (fun x) x] (0.5-d, 0.5+d)

In [ ]:
toRenderable
  $ let d = 5e-9 
        a = 4.5  
     in plotFunctions [incompleteBeta a a] (0.5 - d, 0.5 + d)

In [ ]:
toRenderable
  $ let d = 1e-8
        a = 4.5  
     in plotFunctions [invIncompleteBeta a a] (0.5 - d, 0.5 + d)